In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_hdf('../input/train.h5')
train.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,1.0,NaN,NaN,NaN,1,NaN,104,6.0,9.0,18,1438873,2019-06-07 15:32:01,8,2135019403,0,2329670524,601
1,135939,893,NaN,NaN,NaN,NaN,1,NaN,19,6.0,8.0,0,1185582,2019-06-08 19:40:40,4,2782306428,1,2864801071,1000
2,399254,821,NaN,760.0,NaN,360.0,1,NaN,559,NaN,8.0,0,1555716,2019-06-06 23:59:13,0,1392806005,2,628911675,696
3,68983,1004,1.0,2214.0,NaN,1080.0,0,NaN,129,2.0,8.0,0,1093419,2019-06-09 09:00:12,0,3562553457,3,1283809327,753
4,288999,1076,1.0,2280.0,NaN,1080.0,1,1.0,64,2.0,8.0,0,1400089,2019-06-07 08:28:13,5,2364522023,4,1510695983,582


In [3]:
test1 = pd.read_hdf('../input/test1.h5')
test1.head()

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,osv,package,sid,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,317625,1181,1.0,2196.0,2.0,1080.0,1.0,639,2.0,8.0,188,1440682,2019-06-07 09:42:30,7,1672223856,57,3872258917,658
1,435108,944,3.0,2280.0,3.0,1080.0,1.0,704,6.0,8.0,221,1606824,2019-06-05 20:53:56,3,3767901757,23,129322164,943
2,0,1106,1.0,NaN,NaN,NaN,NaN,39,2.0,5.0,1562,1774642,2019-06-04 10:07:42,0,454638703,30,4226678391,411
3,451504,761,1.0,1344.0,NaN,720.0,NaN,54,2.0,7.0,9,1742535,2019-06-05 01:03:22,0,1507622951,65,3355419572,848
4,0,1001,1.0,665.0,NaN,320.0,1.0,29,5.0,8.0,4,1689686,2019-06-05 08:15:54,0,4116351093,148,2644467751,411


In [4]:
concat_data = pd.concat([train, test1],
                        ignore_index=True)
concat_data.head()

,android_id,apptype,carrier,cus_type,dev_height,dev_ppi,dev_width,fea1_hash,fea_hash,label,lan,location,media_id,ntt,osv,package,sid,timestamp,version
0,316361,1199,1.0,601,NaN,NaN,NaN,2329670524,2135019403,1.0,NaN,0,104,6.0,9.0,18,1438873,2019-06-07 15:32:01,8
1,135939,893,NaN,1000,NaN,NaN,NaN,2864801071,2782306428,1.0,NaN,1,19,6.0,8.0,0,1185582,2019-06-08 19:40:40,4
2,399254,821,NaN,696,760.0,NaN,360.0,628911675,1392806005,1.0,NaN,2,559,NaN,8.0,0,1555716,2019-06-06 23:59:13,0
3,68983,1004,1.0,753,2214.0,NaN,1080.0,1283809327,3562553457,0.0,NaN,3,129,2.0,8.0,0,1093419,2019-06-09 09:00:12,0
4,288999,1076,1.0,582,2280.0,NaN,1080.0,1510695983,2364522023,1.0,1.0,4,64,2.0,8.0,0,1400089,2019-06-07 08:28:13,5


In [5]:
concat_data.shape

(650000, 19)

In [6]:
concat_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650000 entries, 0 to 649999
Data columns (total 19 columns):
android_id    650000 non-null object
apptype       650000 non-null int16
carrier       565574 non-null float64
cus_type      650000 non-null int16
dev_height    510787 non-null float16
dev_ppi       151376 non-null float16
dev_width     510790 non-null float16
fea1_hash     650000 non-null int64
fea_hash      650000 non-null object
label         500000 non-null float64
lan           411171 non-null float64
location      650000 non-null int16
media_id      650000 non-null object
ntt           623201 non-null float16
osv           641446 non-null float64
package       650000 non-null object
sid           650000 non-null object
timestamp     650000 non-null object
version       650000 non-null int64
dtypes: float16(4), float64(4), int16(3), int64(2), object(6)
memory usage: 68.2+ MB


In [7]:
concat_data['apptype'].unique()

array([1199,  893,  821, 1004, 1076,  788, 1106, 1001,  761,  869, 1223,
        917,  941,  938, 1052, 1055,  845,  758, 1211,  929,  884,  740,
        956,  719,  908, 1028,  806, 1007,  764, 1034, 1172, 1082,  743,
        980, 1064,  746, 1193,  716,  767, 1010,  947, 1139, 1067, 1235,
        779,  722,  734,  944, 1088,  974, 1124, 1238, 1169,  704,  923,
       1043,  959, 1130,  899,  851,  827,   95,  776,  713,  833, 1181,
       1019,  782,  803, 1136,  794, 1232, 1229,  905, 1115, 1241,  854,
        728, 1112,  989, 1190, 1031, 1085,  911, 1178, 1100, 1046, 1097,
        857], dtype=int64)

In [8]:
concat_data['carrier'] = concat_data['carrier'].fillna(-999.).astype(str)
print(concat_data['carrier'].unique())
concat_data = pd.get_dummies(concat_data['carrier']).reset_index(drop=True)

['1.0' '-999.0' '3.0' '2.0']


In [9]:
# dummies_list = ['carrier', 'apptype', 'lan', 'ntt', 'osv', 'version', 'cus_type']
# for i in dummies_list:
#     concat_data = pd.get_dummies(concat_data[i], dummy_na=True).reset_index(drop=True)